# Homework #3
Josh Cohen

In [1]:
import sympy as sym
import numpy as np
%matplotlib notebook
from matplotlib import pyplot as plt
from IPython.display import Image
import pandas as pd

## Problem 6.5

## Problem 6.10

## Probelm 6.13

Compare efficacy of Softmax vs Perceptron cost functions in terms of minimum number of misclassifications by pursuing gradient descent based minimization

In [61]:
bc_df = pd.read_csv("breast_cancer_dataset.csv", header=None, index_col=None).dropna(axis=0)


In [118]:
yp = sym.symbols('y_p')
xp_tilde = sym.MatrixSymbol('\widetilde{x_p}', bc_df.shape[1], 1).as_explicit()
w_tilde = sym.MatrixSymbol('\widetilde{w}', bc_df.shape[1], 1).as_explicit()

In [119]:
#percep
cost = sym.Matrix([sym.log(1 + sym.exp(-yp * xp_tilde.T * w_tilde)[0])])
grad_cost = cost.jacobian(w_tilde)
cost_lam = sym.lambdify([yp, xp_tilde, w_tilde], cost)
grad_cost_lam = sym.lambdify([yp, xp_tilde, w_tilde],grad_cost)
# grad2_cost = grad_cost.jacobian(w_tilde)
# grad2_cost_lam = sym.lambdify([yp, xp_tilde, w_tilde], grad2_cost)

In [120]:
def p_cost(w_tilde):
    cost_sum = 0
    for i in range(bc_df.shape[0]):
        cost_sum += cost_lam(bc_df.iloc[i,-1], np.array([1,*bc_df.iloc[i, :-1].to_numpy()]), w_tilde)  * 1/bc_df.shape[0]
        
    ans = np.squeeze(cost_sum,axis=1)
    return ans

In [121]:
def grad_p_cost(w_tilde):
    grad_cost_sum = np.zeros(bc_df.shape[1])
    for i in range(bc_df.shape[0]):
#         thing = grad_cost_lam(bc_df.iloc[i,-1],np.array([1,*bc_df.iloc[i, :-1].to_numpy()]), w_tilde)
#         print(thing.shape)  
        grad_cost_sum += np.squeeze(grad_cost_lam(bc_df.iloc[i,-1],np.array([1,*bc_df.iloc[i, :-1].to_numpy()]), w_tilde), axis=0) * 1/bc_df.shape[0]
#         print(grad_cost_sum.shape)
    return grad_cost_sum 

In [127]:
def grad_2_p_cost(w_tilde):
    grad_2_cost_sum = np.zeros((bc_df.shape[1],bc_df.shape[1]))
    for i in range(bc_df.shape[0]):
#         print(grad_2_cost_sum)
        grad_2_cost_sum += grad2_cost_lam(bc_df.iloc[i,-1] , np.array([1,*bc_df.iloc[i, :-1].to_numpy()]) , w_tilde)
    
    return grad_2_cost_sum * 1/bc_df.shape[0]

In [130]:
def grad_desc(w_last):
    eps = 10e-3
    cost_vals = np.zeros(3000)
    num_its = 0
    cost_vals[0] = p_cost(w_last)
    grad_last = grad_p_cost(w_last)
#     grad_2_last = grad_2_p_cost(w_last)
    num_its += 1

    while(np.linalg.norm(grad_last) > eps):
#         w_next = w_last - np.matmul(np.linalg.inv(grad_2_last), grad_last)
        w_next = w_last - .1 *grad_last
        num_its += 1
        if (num_its > 1000):
            break;
        
        w_last = w_next
        cost_vals[num_its] = p_cost(w_last)
        grad_last = grad_p_cost(w_last)
#         grad_2_last = grad_2_p_cost(w_last)
        print(np.linalg.norm(grad_last))
    
    
    return w_last, cost_vals, num_its

In [ ]:
w_tilde = np.array([10, 1, 1, 1, 10, 1, 1, 1, 1, 1])
w, costs, its = grad_desc(w_tilde) 


In [ ]:
w_last = np.array([10, 1, 1, 1, 10, 1, 1, 1, 1, 1])
grad_last = grad_p_cost(w_last)
grad_2_last = grad_2_p_cost(w_last)
np.matmul(np.linalg.inv(grad_2_last), grad_last)

In [ ]:
w_last = np.array([10, 1, 1, 1, 1, 1, 1, 1, 1, 1])

grad_last = np.expand_dims(grad_p_cost(w_last),1)
grad_2_last = grad_2_p_cost(w_last)
print(grad_2_last)
np.matmul(grad_2_last.T, grad_last)

In [148]:
def misclass_calc_softmax(w_tilde):
    miscalc = 0
    for i in range(bc_df.shape[0]):
        xp_tilde = np.array([1,*bc_df.iloc[i, :-1].to_numpy()])
        yp = bc_df.iloc[i,-1]
        if np.sign(np.matmul(xp_tilde.T, w_tilde)) != np.sign(yp):
            miscalc +=1
    
    return miscalc

In [149]:
misclass_calc_softmax(w)

20

In [152]:
def missclass_calc_perceptron(w_tilde):
    miscalc = 0
    for i in range(bc_df.shape[0]):
        xp_tilde = np.array([1,*bc_df.iloc[i, :-1].to_numpy()])
        yp = bc_df.iloc[i,-1]
        if np.maximum(0, -yp * np.matmul(xp_tilde, w_tilde)) != 0:
            miscalc += 1
        
    return miscalc

In [153]:
missclass_calc_perceptron(w)

20

## Problem 6.15

In [301]:
cd_df = pd.read_csv("credit_dataset.csv", header=None, index_col=None).dropna(axis=0)

In [302]:
cd_mean = cd_df.iloc[:-1,:].mean(axis=1)
cd_std = cd_df.iloc[:-1,:].std(axis=1)
cd_std_norm = cd_df.iloc[:-1,:].sub(cd_mean, axis=0).div(cd_std, axis=0)

In [303]:
def setup_symbols(df, axis):
    N = df.shape[axis]
    yp = sym.symbols('y_p')
    xp_tilde = sym.MatrixSymbol('\widetilde{x_p}', N, 1).as_explicit()
    w_tilde = sym.MatrixSymbol('\widetilde{w}', N, 1).as_explicit()
    
    cost = sym.Matrix([sym.log(1 + sym.exp(-yp * xp_tilde.T * w_tilde)[0])])
    grad_cost = cost.jacobian(w_tilde)
    cost_lam = sym.lambdify([yp, xp_tilde, w_tilde], cost)
    grad_cost_lam = sym.lambdify([yp, xp_tilde, w_tilde],grad_cost)
#     grad2_cost = grad_cost.jacobian(w_tilde)
#     grad2_cost_lam = sym.lambdify([yp, xp_tilde, w_tilde], grad2_cost)
    
    return cost_lam, grad_cost_lam, grad2_cost_lam

In [308]:
#setup specific cost functions and link to df
cost_lam, grad_cost_lam, grad2_cost_lam = setup_symbols(cd_df, 0)
xp_tildes = cd_std_norm
yps = cd_df.iloc[-1,:].to_numpy()
df = cd_df
N = df.shape[0]
P = df.shape[1]

In [313]:
def cost(w_tilde):
    cost_sum = 0
#     print("AHHHHH")
    for i in range(P):
#         print(np.array([1,*xp_tildes.iloc[:,i].to_numpy()]).shape)
        cost_sum += cost_lam(yps[i], np.array([1,*xp_tildes.iloc[:,i].to_numpy()]), w_tilde)  * 1/P
        
    ans = np.squeeze(cost_sum,axis=1)
    return ans

In [314]:
def grad_cost(w_tilde):
    grad_cost_sum = np.zeros(N)
    for i in range(P):
        grad_cost_sum += np.squeeze(grad_cost_lam(yps[i], np.array([1,*xp_tildes.iloc[:,i].to_numpy()]), w_tilde), axis=0) * 1/P
    return grad_cost_sum 

In [315]:
# def grad_2_p_cost(w_tilde):
#     grad_2_cost_sum = np.zeros((cd_df.shape[0],cd_df.shape[0]))
#     for i in range(bc_df.shape[0]):
#         grad_2_cost_sum += grad2_cost_lam(bc_df.iloc[i,-1] , np.array([1,*bc_df.iloc[i, :-1].to_numpy()]) , w_tilde)
    
#     return grad_2_cost_sum * 1/df.shape[1]

In [318]:
def grad_desc(w_last):
    eps = 10e-3
    cost_vals = np.zeros(3000)
    num_its = 0
    cost_vals[0] = cost(w_last)
    grad_last = grad_cost(w_last)
    num_its += 1

    while(np.linalg.norm(grad_last) > eps):
        w_next = w_last - .1 *grad_last
        num_its += 1
        if (num_its > 1000):
            print("hit max its")
            break;
        
        w_last = w_next
        cost_vals[num_its] = cost(w_last)
        grad_last = grad_cost(w_last)
        print(np.linalg.norm(grad_last))
    
    
    return w_last, cost_vals, num_its

In [ ]:
w_tilde = np.ones(21)
w, costs, its = grad_desc(w_tilde)

In [320]:
def confusion_matrix_gen(w_tilde):
    confusion_matrix = pd.DataFrame(np.zeros((2,2)), index=["a_bad", "a_good"], columns=["p_bad", "p_good"]).astype('int')
    for i in range(df.shape[1]):
        xp_tilde = np.array([1,*xp_tildes.iloc[:,i].to_numpy()])
        yp = yps[i]
        #correctly predicted
        if np.sign(np.matmul(xp_tilde.T, w_tilde)) == np.sign(yp):
            #pg, ag
            if (yp > 0):
                confusion_matrix.loc["a_good", "p_good"] += 1
            else:
                confusion_matrix.loc["a_bad", "p_bad"] += 1
                
        else:
            if (yp > 0):
                confusion_matrix.loc["a_good", "p_bad"] += 1
            else:
                confusion_matrix.loc["a_bad", "p_good"] += 1
                
    return confusion_matrix

In [321]:
confusion_matrix = confusion_matrix_gen(w)

In [322]:
confusion_matrix

,p_bad,p_good
a_bad,146,154
a_good,76,624


In [323]:
acurracy = 1 - (confusion_matrix.loc["a_good", "p_bad"] + confusion_matrix.loc["a_bad", "p_good"])/1000

In [324]:
acurracy

0.77

## Probelm 6.16